### Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pySankey.sankey import sankey

pd.set_option("display.max_columns", None)

from datavis_fun import *
from rfm_fun import *

import jupyter_black

jupyter_black.load()

### Load dataset(s)

Data Source: https://www.kaggle.com/datasets/mathchi/online-retail-ii-data-set-from-ml-repository/data

 Description of columns:
 - InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
  
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
  
- Description: Product (item) name. Nominal.
  
- Quantity: The quantities of each product (item) per transaction. Numeric.
  
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.

- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
  
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
  
- Country: Country name. Nominal. The name of the country where a customer resides.

In [ ]:
df1 = pd.read_csv("online_retail_2009-10.csv", sep=",")
df2 = pd.read_csv("online_retail_2010-11.csv", sep=",")

df1 = standardize_colnames(df=df1)
df2 = standardize_colnames(df=df2)

### Merge data in one big dataframe 

In [ ]:
data_all = pd.concat([df1, df2])

### Duplicated values in a merged dataset

In [5]:
show_dup_rows(df=data_all)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
379,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,12/1/2009 11:34,1.95,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,12/1/2009 11:34,1.95,16329.0,United Kingdom
365,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,12/1/2009 11:34,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,12/1/2009 11:34,3.75,16329.0,United Kingdom
363,489517,21912,VINTAGE SNAKES & LADDERS,1,12/1/2009 11:34,3.75,16329.0,United Kingdom
...,...,...,...,...,...,...,...,...
440149,C574510,22360,GLASS JAR ENGLISH CONFECTIONERY,-1,11/4/2011 13:25,2.95,15110.0,United Kingdom
461407,C575940,23309,SET OF 60 I LOVE LONDON CAKE CASES,-24,11/13/2011 11:38,0.55,17838.0,United Kingdom
461408,C575940,23309,SET OF 60 I LOVE LONDON CAKE CASES,-24,11/13/2011 11:38,0.55,17838.0,United Kingdom
529980,C580764,22667,RECIPE BOX RETROSPOT,-12,12/6/2011 10:38,2.95,14562.0,United Kingdom


In [6]:
dd_df = data_all.drop_duplicates(keep="first")
dd_df.shape[0]

1033036

In [7]:
show_dup_rows(dd_df)

No identical rows found.


In [8]:
dd_df.to_csv("online_retail_2009-11.csv", index=False)

### Loading aggregated dataframe 

In [9]:
orig = pd.read_csv("online_retail_2009-11.csv", sep=",")
orig.head(3)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom


In [10]:
dd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033036 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   invoice      1033036 non-null  object 
 1   stockcode    1033036 non-null  object 
 2   description  1028761 non-null  object 
 3   quantity     1033036 non-null  int64  
 4   invoicedate  1033036 non-null  object 
 5   price        1033036 non-null  float64
 6   customer_id  797885 non-null   float64
 7   country      1033036 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 70.9+ MB


 - Change type of invoicedate to datetime
 - Change customerid variable type to object

In [11]:
orig.customer_id = orig.customer_id.astype(str)
format = "%m/%d/%Y %H:%M"
orig["invoicedate"] = pd.to_datetime(orig.invoicedate, format=format)

orig.head(2)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


In [12]:
orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033036 entries, 0 to 1033035
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   invoice      1033036 non-null  object        
 1   stockcode    1033036 non-null  object        
 2   description  1028761 non-null  object        
 3   quantity     1033036 non-null  int64         
 4   invoicedate  1033036 non-null  datetime64[ns]
 5   price        1033036 non-null  float64       
 6   customer_id  1033036 non-null  object        
 7   country      1033036 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 63.1+ MB


### Excluding cancelled orders (merged dataset)

 I also plan to exclude cancelled orders for RFM analysis even though they can be analyzed separately.
 

In [13]:
orig = orig[~orig.invoice.str.contains("C")]

### Price & quantity constraints

Quantity and Price columns should be greater than zero. There can't be negative quantities or prices, considering removal of cancelled orders.  

In [14]:
if orig.price.values.min() <= 0:
    print("There are negative values in 'price' column")

if orig.quantity.values.min() <= 0:
    print("There are negative values in 'quantity' column")

There are negative values in 'price' column
There are negative values in 'quantity' column


Filter that out

In [15]:
orig = orig.loc[(orig.quantity > 0) & (orig.price > 0)]

In [16]:
if orig.price.values.min() <= 0:
    print("There are negative values in 'price' column")

if orig.quantity.values.min() <= 0:
    print("There are negative values in 'quantity' column")

### Missing values (merged dataset)

In [17]:
missing_tab(orig)

,column_name,no_missing,percent_missing


However, there can be still some missing values which are represented as text. I will check object columns by format which is stated in columns description:


In [18]:
orig.tail(3)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
1033033,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1033034,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France
1033035,581587,POST,POSTAGE,1,2011-12-09 12:50:00,18.00,12680.0,France


In [19]:
orig[~orig["customer_id"].astype(str).str.contains(r"^\d{5}", regex=True)]

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
569,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,nan,United Kingdom
570,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1,2009-12-01 11:49:00,0.85,nan,United Kingdom
1027,489548,22271,FELTCRAFT DOLL ROSIE,1,2009-12-01 12:32:00,2.95,nan,United Kingdom
1028,489548,22254,FELT TOADSTOOL LARGE,12,2009-12-01 12:32:00,1.25,nan,United Kingdom
1029,489548,22273,FELTCRAFT DOLL MOLLY,3,2009-12-01 12:32:00,2.95,nan,United Kingdom
...,...,...,...,...,...,...,...,...
1032669,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,nan,United Kingdom
1032670,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,nan,United Kingdom
1032671,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,nan,United Kingdom
1032672,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,nan,United Kingdom


In [20]:
orig = orig[orig["customer_id"].str.contains(r"^\d{5}", regex=True)]

In [21]:
orig[~orig["invoice"].str.contains(r"^\d{6}", regex=True)]

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country


In [22]:
orig.stockcode.nunique()

4631

In [23]:
orig.description.nunique()

5283

In [24]:
orig.sort_values(["quantity", "price"])

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
61632,494914,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-01-19 17:04:00,0.001,16705.0,United Kingdom
73934,496222,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-01-29 13:53:00,0.001,13583.0,United Kingdom
76862,496473,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-01 15:38:00,0.001,17350.0,United Kingdom
78926,496643,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-03 11:58:00,0.001,13408.0,United Kingdom
89790,497935,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-15 10:47:00,0.001,13408.0,United Kingdom
...,...,...,...,...,...,...,...,...
125789,501534,21099,SET/6 STRAWBERRY PAPER CUPS,12960,2010-03-17 13:09:00,0.100,13902.0,Denmark
125791,501534,21091,SET/6 WOODLAND PAPER PLATES,12960,2010-03-17 13:09:00,0.100,13902.0,Denmark
89849,497946,37410,BLACK AND WHITE PAISLEY FLOWER MUG,19152,2010-02-15 11:57:00,0.100,13902.0,Denmark
557400,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.040,12346.0,United Kingdom


### Checking condition that each stockid corresponds to one description (yearly)

In [25]:
gdf = orig.copy()
subdf = gdf[["stockcode", "description"]]

# Table with number of stockcodes per description
print("Initial Number of Unique Descriptions:{}".format(gdf.description.unique().size))
print("Initial Number of Unique Stockcodes: {}".format(gdf.stockcode.unique().size))

print("-----")
print("Step 1: Find Descriptions That Have More than 1 Stockcode and Delete Them\n")
df_desc = subdf.drop_duplicates()
oned_mcodes = (
    df_desc.groupby(["description"])
    .agg(n_stockcodes=("stockcode", "count"))
    .sort_values("n_stockcodes", ascending=False)
    .reset_index()
)
oned_mcodes = oned_mcodes.loc[oned_mcodes.n_stockcodes > 1]

# Store all descriptions with more than one stockcode in a list
d_todel = oned_mcodes.description.tolist()

print("Number of descriptions with more than one stockcode: {}".format(len(d_todel)))
print(
    "Expected Number of Unique Desc after Filtering: {}".format(
        gdf.description.unique().size - len(d_todel)
    )
)

# Filter dataset out
gdf = gdf.loc[~gdf.description.isin(d_todel)]

print(
    "Actual Number of Unique Descriptions After Filtering: {}".format(
        gdf.description.unique().size
    )
)
print("-----")
print("Step 2: Find Stockcodes That Have More than 1 Description and Delete Them\n")

# Reassign subdf
subdf = gdf[["stockcode", "description"]]

print("Step 1 (Revisited):")
print(
    "Number of Unique Stockcodes After Step 1: {}".format(gdf.stockcode.unique().size)
)
print(
    "Number of Unique Descriptions After Step 1: {}\n".format(
        gdf.description.unique().size
    )
)

# Table which  number of descriptions per stockcode
df_stock = subdf.drop_duplicates()
onec_mdesc = (
    df_stock.groupby(["stockcode"])
    .agg(n_desc=("description", "count"))
    .sort_values("n_desc", ascending=False)
    .reset_index()
)
onec_mdesc = onec_mdesc.loc[onec_mdesc.n_desc > 1]
c_todel = onec_mdesc.stockcode.tolist()

print("Number of Stockcodes w More than One Desc: {}".format(len(c_todel)))
print(
    "Expected Number of Stockcodes After Filtering: {}".format(
        gdf.stockcode.unique().size - len(c_todel)
    )
)

gdf = gdf.loc[~gdf.stockcode.isin(c_todel)]

print(
    "Actual Number of Stockcodes After Filtering: {}".format(
        gdf.stockcode.unique().size
    )
)
print("-----")
print("Results:\n")
print("Final Number of Unique Stockcodes: {}".format(gdf.stockcode.unique().size))
print("Final Number of Unique Descriptions: {}\n".format(gdf.description.unique().size))

data_all_fixed = gdf

Initial Number of Unique Descriptions:5283
Initial Number of Unique Stockcodes: 4631
-----
Step 1: Find Descriptions That Have More than 1 Stockcode and Delete Them

Number of descriptions with more than one stockcode: 29
Expected Number of Unique Desc after Filtering: 5254
Actual Number of Unique Descriptions After Filtering: 5254
-----
Step 2: Find Stockcodes That Have More than 1 Description and Delete Them

Step 1 (Revisited):
Number of Unique Stockcodes After Step 1: 4582
Number of Unique Descriptions After Step 1: 5254

Number of Stockcodes w More than One Desc: 613
Expected Number of Stockcodes After Filtering: 3969
Actual Number of Stockcodes After Filtering: 3969
-----
Results:

Final Number of Unique Stockcodes: 3969
Final Number of Unique Descriptions: 3969



In [26]:
data_all_fixed.to_csv("retail_clean_2009-11.csv", index=False)

In [29]:
data_all_fixed

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1033028,581587,22629,SPACEBOY LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680.0,France
1033030,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
1033033,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1033034,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France
